In [ ]:
using Plots, LaTeXStrings
using Random

In [ ]:
include("../src/splines.jl")
include("../src/poisson_solver_splines.jl")
include("../src/bump_on_tail_distributions.jl")
include("../src/sampling.jl")
include("../src/time_marching.jl")
include("../src/h5routines.jl")
include("../src/regression.jl")
include("../src/visualisation.jl");

In [ ]:
nₚ₁ = 10
nₚ₂ = 1
nₚ₃ = 1
nₚ₄ = 1
nₚ₅ = 1

IP = IntegratorParameters(1e-1, 250, 251, nₚ₁*nₚ₂*nₚ₃*nₚ₄*nₚ₅, 16, Int(5e3))
#                           dt   nₜ   nₛ          nₚ            nₕ    Nₚ

κₘᵢₙ  = 0.1;  κₘₐₓ  = 0.5;
εₘᵢₙ  = 0.03; εₘₐₓ  = 0.03;
aₘᵢₙ  = 0.1;  aₘₐₓ  = 0.1;
v₀ₘᵢₙ = 4.5;  v₀ₘₐₓ = 4.5;
σₘᵢₙ  = 0.5;  σₘₐₓ  = 0.5;

κₛₐₘₚ = 0.3; εₛₐₘₚ = 0.03; aₛₐₘₚ = 0.1; v₀ₛₐₘₚ= 4.5; σₛₐₘₚ = 0.5

#ζₘₐₓ = (κₛₐₘₚ/κₘᵢₙ); ζₘᵢₙ = (κₛₐₘₚ/κₘₐₓ)
#         ζ = λ*ζₘᵢₙ + (1-λ)*ζₘₐₓ
#         μ[i,1] = κₛₐₘₚ/ζ

μ = zeros(IP.nₚ, 5)
for i in 1:IP.nₚ
    μ[i,:] = [κₘᵢₙ, εₛₐₘₚ, aₛₐₘₚ, v₀ₛₐₘₚ, σₛₐₘₚ]
end

for i in 1:IP.nₚ
    μ[i,1] += (κₘₐₓ - κₘᵢₙ)*(i-1)/(IP.nₚ-1)
end

# for i in 1:IP.nₚ
#     μ[i,:] = [κₘᵢₙ + rand(1)[1] * (κₘₐₓ - κₘᵢₙ), 
#             εₘᵢₙ + rand(1)[1] * (εₘₐₓ - εₘᵢₙ), 
#             aₘᵢₙ + rand(1)[1] * (aₘₐₓ - aₘᵢₙ),
#             v₀ₘᵢₙ + rand(1)[1] * (v₀ₘₐₓ - v₀ₘᵢₙ),
#             σₘᵢₙ + rand(1)[1] * (σₘₐₓ - σₘᵢₙ)]
# end

# wave number κ, amplitude ε, tail percentage, tail mean v₀, tail sd σ
μₛₐₘₚ = [κₛₐₘₚ, εₛₐₘₚ, aₛₐₘₚ, v₀ₛₐₘₚ, σₛₐₘₚ]

μ

In [ ]:
χ = μ[:,1] ./ κₛₐₘₚ

In [ ]:
const S = PBSpline(3, IP.nₕ, 2π/μₛₐₘₚ[1]);

In [ ]:
Random.seed!(1234)

In [ ]:
# Reference draw
P₀ = draw_g_bumpontail_accept_reject(IP.Nₚ, fₓ, μₛₐₘₚ);

In [ ]:
@time IC = IntegratorCache(IP);

In [ ]:
K = stiffnessmatrix_PBSpline(S)
K_ = zero(K); K_ .= K; K_[S.nₕ,:] = ones(S.nₕ);

In [ ]:
@time Result = integrate_vp(P₀, S, μ, μₛₐₘₚ, K_, IP, IC; save=true, given_phi = false);

In [ ]:
#  13.924736 seconds (43.08 M allocations: 5.875 GiB, 6.93% gc time)
#  8.762504 seconds (680.00 k allocations: 35.518 MiB, 0.32% gc time)   # eval_deriv_PBSBasis
#  8.854784 seconds (601.51 k allocations: 31.372 MiB)                  # eval_PBSBasis

In [ ]:
save_h5("../runs/BoT_Np5e4_k_010_050_np_10_T25.h5", IP, S, μₛₐₘₚ, μ, Result);

In [ ]:
W = zero(Result.Φ[1,:]);

for i in eachindex(W)
    W[i] = 0.5 * dot(Result.Φ[:,i], K*Result.Φ[:,i])
end

W = reshape(W, (IP.nₛ,IP.nₚ));

for p in 1:IP.nₚ
    W[:,p] .*= χ[p]^2
end

In [ ]:
t1 = collect(range(0, stop=IP.dt*IP.nₜ, length=IP.nₛ));

In [ ]:
plot(t1, W[:,:], linewidth = 2, xlabel = L"$n_t$", yscale = :log10, legend = :none,
     grid = true, gridalpha = 0.5, minorgrid = true, minorgridalpha = 0.2)

IC = 0
Result = 0
GC.gc()

In [ ]:
α, β = get_regression_αβ(t1, W, 2)
β

In [ ]:
Wₗᵢₙ = zero(W)
for i in 1:size(W,2)
    Wₗᵢₙ[:,i] .= exp.(α[i] .+ β[i] .* t1)
end

In [ ]:
plot(xlabel = L"$n_t$", yscale = :log10, ylims = (1E-3,1E1), legend = :none,
     grid = true, gridalpha = 0.5, minorgrid = true, minorgridalpha = 0.2)
plot!(t1, W[:,1:5], linewidth = 2, alpha = 0.25)
plot!(t1, Wₗᵢₙ[:,1:5], linewidth = 2, alpha = 0.5)

In [ ]:
plot(xlabel = L"$n_t$", yscale = :log10, ylims = (1E-3,1E1), legend = :none,
     grid = true, gridalpha = 0.5, minorgrid = true, minorgridalpha = 0.2)
plot!(t1, W[:,6:10], linewidth = 2, alpha = 0.25)
plot!(t1, Wₗᵢₙ[:,6:10], linewidth = 2, alpha = 0.5)